<a href="https://colab.research.google.com/github/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/blob/main/%E5%B0%88%E9%A1%8C%E5%A0%B1%E5%91%8A_Michael_Jackson_%E5%B0%88%E8%BC%AF%E4%B8%BB%E9%A1%8C%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 專題報告 - Michael Jackson 專輯主題機器人

**一、專案目的**

本專案旨在利用 LINE Messaging API 建立一個以 Michael Jackson 音樂作品為主題的互動式 LINE 聊天機器人。使用者可以透過指令查詢專輯資訊、進行歌曲搜尋、投票選擇最喜愛的專輯，並即時查看排行榜結果。

本人非常喜愛 Michael Jackson 的音樂作品，其跨世代的影響力與經典曲目在全球廣受歡迎。透過本次專題，不僅得以整理與呈現其完整專輯資料，也希望藉此設計出一個能夠讓粉絲及使用者輕鬆互動、回味經典的 LINE 聊天機器人。

**二、功能概述**
1. 主選單功能 (Carousel 選單)
- 觸發指令：menu
- 透過 Carousel Template 顯示所有專輯清單 (目前共 11 張專輯)
- 每張卡片提供兩個選項：
  - View Info：查詢專輯完整資訊（年份、銷量、曲目）
  - Vote：對該專輯進行投票
2. 專輯資訊查詢
- 觸發指令：info {專輯名稱} 或透過 Carousel 點擊
- 顯示指定專輯之：年份、總銷售量 (格式：70,000,000 (70M))、完整曲目
3. 投票機制
- 觸發指令：vote {專輯名稱} 或透過 Carousel 點擊
- 為專輯增加 1 票，並顯示目前累積投票
4. 排行榜統計
- 觸發指令：/top
- 依投票數排序回傳排行榜
5. 歌曲查詢 (反向歌曲索引功能)
- 輸入任意歌曲名稱自動對應所屬專輯，並顯示專輯資訊
6. 例外處理 (Fallback)
- 輸入無法辨識之指令時提示 menu 與歌曲查詢指引

**三、技術架構**

- 開發平台：Google Colab (含 Ngrok 整合)
- 伺服器框架：Flask (Webhook Server)
- LINE SDK：line-bot-sdk v3 (新版 APIClient 實作)
- 公開隧道：pyngrok + ngrok authtoken (Colab Secret 管理)
- 憑證管理：透過 os.getenv() 讀取 Colab Secrets


In [ ]:

!pip install line-bot-sdk ngrok pyngrok iPython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:

from google.colab import userdata
from pyngrok import ngrok
from linebot.v3.webhook import WebhookHandler
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import MessagingApi, Configuration, TextMessage, ReplyMessageRequest
from linebot.v3.messaging.models import TemplateMessage, CarouselTemplate, CarouselColumn, MessageAction
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import ApiClient

# Colab secrets for credentials
CHANNEL_ACCESS_TOKEN = userdata.get("LINE_CHANNEL_ACCESS_TOKEN")
CHANNEL_SECRET = userdata.get("LINE_CHANNEL_SECRET")
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")


In [ ]:

# Apply ngrok authentication
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


**四、資料設計**

目前共收錄 Michael Jackson 之 11 張正式專輯資料，包含 Motown 早期、主流與身後專輯。
資料欄位：
- year：年份
- sales：銷量 (整數，利於格式化)
- songs：曲目
- cover：封面 (部分使用 placeholder)


In [ ]:

album_data = {
    "Got to Be There": {
        "year": 1972, "sales": 2000000,
        "songs": [
            "Ain't No Sunshine", "I Wanna Be Where You Are", "Girl Don't Take Your Love From Me",
            "In Our Small Way", "Got to Be There", "Rockin' Robin", "Wings of My Love",
            "Maria (You Were the Only One)", "Love Is Here and Now You're Gone", "You've Got a Friend"
        ],
        "cover": "https://upload.wikimedia.org/wikipedia/en/thumb/b/b2/Mj1971-got-to-be-there.jpg/250px-Mj1971-got-to-be-there.jpg"
    },
    "Ben": {
        "year": 1972, "sales": 2000000,
        "songs": [
            "Ben", "Greatest Show on Earth", "People Make the World Go 'Round", "We've Got a Good Thing Going",
            "Everybody's Somebody's Fool", "My Girl", "What Goes Around Comes Around", "In Our Small Way",
            "Shoo-Be-Doo-Be-Doo-Da-Day", "You Can Cry on My Shoulder"
        ],
        "cover": "https://lh3.googleusercontent.com/qx2XfantCerpCCvMMJRHWulhaj73WDavPph7aB19EG-mDsbIpRk6prjQxVhl6BOmX1RVFP_l-uoNX_Akyg=w544-h544-s-l90-rj"
    },
    "Music & Me": {
        "year": 1973, "sales": 1000000,
        "songs": [
            "With a Child's Heart", "Up Again", "All the Things You Are", "Happy", "Too Young",
            "Doggin' Around", "Johnny Raven", "Euphoria", "Morning Glow", "Music and Me"
        ],
        "cover": "https://lh3.googleusercontent.com/33_r_7savr68acaP1Df50BpQyv_ZO3uZIMT-BAASqmswcXF_9xXupUifxNzxY6Ia1nXyL5VmOIfvxgAhDA=w544-h544-l90-rj"
    },
    "Forever, Michael": {
        "year": 1975, "sales": 1000000,
        "songs": [
            "We're Almost There", "Take Me Back", "One Day in Your Life", "Cinderella Stay Awhile",
            "We've Got Forever", "Just a Little Bit of You", "You Are There", "Dapper-Dan",
            "Dear Michael", "I'll Come Home to You"
        ],
        "cover": "https://lh3.googleusercontent.com/CCP03PdfzD_z30UlbwV6fqAoPivZZNH445W-avgxcx63hrCHZlXNDul3Ze9Sd0OVVbn0Ic1dH0FSMYQjsQ=w544-h544-s-l90-rj"
    },
    "Thriller": {
        "year": 1982, "sales": 70000000,
        "songs": [
            "Wanna Be Startin' Somethin'", "Baby Be Mine", "The Girl Is Mine", "Thriller",
            "Beat It", "Billie Jean", "Human Nature", "P.Y.T. (Pretty Young Thing)", "The Lady in My Life"
        ],
        "cover": "https://upload.wikimedia.org/wikipedia/en/5/55/Michael_Jackson_-_Thriller.png"
    },
    "Bad": {
        "year": 1987, "sales": 35000000,
        "songs": [
            "Bad", "The Way You Make Me Feel", "Speed Demon", "Liberian Girl", "Just Good Friends",
            "Another Part of Me", "Man in the Mirror", "I Just Can't Stop Loving You", "Dirty Diana",
            "Smooth Criminal", "Leave Me Alone"
        ],
        "cover": "https://upload.wikimedia.org/wikipedia/en/5/51/Michael_Jackson_-_Bad.png"
    },
    "Dangerous": {
        "year": 1991, "sales": 32000000,
        "songs": [
            "Jam", "Why You Wanna Trip on Me", "In the Closet", "She Drives Me Wild", "Remember the Time",
            "Can't Let Her Get Away", "Heal the World", "Black or White", "Who Is It", "Give In to Me",
            "Will You Be There", "Keep the Faith", "Gone Too Soon", "Dangerous"
        ],
        "cover": "https://lh3.googleusercontent.com/acFvHA1OEoI0HBPPG33zidd9n9aG1OTvo7XQQeFjEeQObGv6R3464BvFijHerp3Sit5UeHvQnx6LMoE=w544-h544-l90-rj"
    },
    "HIStory": {
        "year": 1995, "sales": 20000000,
        "songs": [
            "Scream", "They Don't Care About Us", "Stranger in Moscow", "This Time Around", "Earth Song",
            "D.S.", "Money", "Come Together", "You Are Not Alone", "Childhood", "Tabloid Junkie",
            "2 Bad", "HIStory", "Little Susie", "Smile"
        ],
        "cover": "https://lh3.googleusercontent.com/R7pwf7-lcPvK3dxv8jMkUd4SlbVmrM-nZOFEJqnHGLQFBl4lqj1gyeWSFO5X9HZxgUhTh4KM8n0l7j_k=w544-h544-l90-rj"
    },
    "Invincible": {
        "year": 2001, "sales": 6000000,
        "songs": [
            "Unbreakable", "Heartbreaker", "Invincible", "Break of Dawn", "Heaven Can Wait", "You Rock My World",
            "Butterflies", "Speechless", "2000 Watts", "You Are My Life", "Privacy", "Don't Walk Away",
            "Cry", "The Lost Children", "Whatever Happens", "Threatened"
        ],
        "cover": "https://lh3.googleusercontent.com/L0nwBz3JF8kFa1-1PAOiLC6pB49lsRc3QRCbj1gvU3uCNf4SXnLUGtAFBha1CZ02mhQzMJrcl-rJ0pSG=w544-h544-l90-rj"
    },
    "Michael": {
        "year": 2010, "sales": 1000000,
        "songs": [
            "Hold My Hand", "Hollywood Tonight", "Keep Your Head Up", "I Like the Way You Love Me",
            "Monster", "Best of Joy", "Breaking News", "(I Can't Make It) Another Day", "Behind the Mask", "Much Too Soon"
        ],
        "cover": "https://lh3.googleusercontent.com/-DE1fXN4FdvCXTfQz9N6TMX1qhjyIOYO5rRlFpWpZxBPWLH2un4Ia_yAuuZwXgtNEJPdeWvW5cBfL_ct=w544-h544-l90-rj"
    },
    "Xscape": {
        "year": 2014, "sales": 1000000,
        "songs": [
            "Love Never Felt So Good", "Chicago", "Loving You", "A Place with No Name", "Slave to the Rhythm",
            "Do You Know Where Your Children Are", "Blue Gangsta", "Xscape"
        ],
        "cover": "https://lh3.googleusercontent.com/qk8rEQ5zgdeXxszR37SjJHOPOlPG6NhD5F4r-rxdxdrdmf7rF6xv_OdrCBuZBxaTFc4fVK2ImJLW5FKt=w544-h544-l90-rj"
    }
}
vote_counter = {album: 0 for album in album_data}
song_lookup = {song.lower(): album for album, data in album_data.items() for song in data["songs"]}


In [ ]:

from flask import Flask, request, abort

app = Flask(__name__)
configuration = Configuration(access_token=CHANNEL_ACCESS_TOKEN)
api_client = ApiClient(configuration)
line_bot_api = MessagingApi(api_client)
handler = WebhookHandler(CHANNEL_SECRET)

def format_sales(sales):
    return f"{sales:,} ({sales//1_000_000}M)"

def build_album_carousel():
    columns = []
    for album_name, data in album_data.items():
        column = CarouselColumn(
            thumbnail_image_url=data['cover'],
            title=album_name,
            text=f"{data['year']} - {format_sales(data['sales'])}",
            actions=[
                MessageAction(label="View Info", text=f"info {album_name}"),
                MessageAction(label="Vote", text=f"vote {album_name}")
            ]
        )
        columns.append(column)
    carousel_template = CarouselTemplate(columns=columns[:10])
    return TemplateMessage(alt_text="Album Selection", template=carousel_template)

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=False).decode('utf-8')
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent)
def handle_message(event):
    if isinstance(event.message, TextMessageContent):
        user_text = event.message.text.strip()

        if user_text.lower() == 'menu':
            carousel_msg = build_album_carousel()
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[carousel_msg])
            )
            return

        if user_text.lower().startswith("info "):
            album_name = user_text[5:].strip()
            if album_name in album_data:
                data = album_data[album_name]
                songs = "\n".join(f"- {song}" for song in data["songs"])
                reply = f"🎶 Album: {album_name}\n📅 Year: {data['year']}\n💿 Sales: {format_sales(data['sales'])}\n🎵 Songs:\n{songs}"
            else:
                reply = "❌ Album not found."
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[TextMessage(text=reply)])
            )
            return

        if user_text.lower().startswith("vote "):
            album_name = user_text[5:].strip()
            if album_name in vote_counter:
                vote_counter[album_name] += 1
                reply = f"✅ Voted for {album_name}! Total votes: {vote_counter[album_name]}"
            else:
                reply = "❌ Album not found for voting."
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[TextMessage(text=reply)])
            )
            return

        if user_text.lower() == "/top":
            sorted_votes = sorted(vote_counter.items(), key=lambda x: x[1], reverse=True)
            leaderboard = "\n".join(f"{i+1}. {name} - {votes} votes" for i, (name, votes) in enumerate(sorted_votes))
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[TextMessage(text=leaderboard)])
            )
            return

        song_key = user_text.lower()
        if song_key in song_lookup:
            album_name = song_lookup[song_key]
            data = album_data[album_name]
            songs = "\n".join(f"- {song}" for song in data["songs"])
            reply = f"🎶 Album (from song '{user_text}'): {album_name}\n📅 Year: {data['year']}\n💿 Sales: {format_sales(data['sales'])}\n🎵 Songs:\n{songs}"
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[TextMessage(text=reply)])
            )
            return

        reply = "🤖 Type 'menu' to browse albums, or enter song name."
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(reply_token=event.reply_token, messages=[TextMessage(text=reply)])
        )


In [ ]:

public_url = ngrok.connect(5000).public_url
print("Your public URL to paste into LINE webhook:", public_url + "/callback")
app.run(port=5000)


Your public URL to paste into LINE webhook: https://b4a5-34-48-183-192.ngrok-free.app/callback
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 02:16:39] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 02:16:59] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 02:17:06] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 02:17:12] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 02:17:23] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 02:17:24] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 02:17:28] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 02:17:37] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 02:17:37] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 02:17:39

In [ ]:
from IPython.display import display, HTML

display(HTML("""
<div style="display:flex;">
  <img src="https://github.com/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/raw/4839379d64825e8a4e98eb67daa66980e836d859/MJ%204.jpg" width="300">
  <img src="https://github.com/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/raw/4839379d64825e8a4e98eb67daa66980e836d859/MJ%202.jpg" width="300">
  <img src="https://github.com/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/raw/4839379d64825e8a4e98eb67daa66980e836d859/MJ%201.jpg" width="300">
  <img src="https://github.com/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/raw/4839379d64825e8a4e98eb67daa66980e836d859/MJ%203.jpg" width="300">
</div>
"""))

**五、目前功能限制與改進方向**

在目前版本中，系統於技術層面尚有部分優化空間。目前排行榜功能僅以純文字方式呈現，未來可透過 Flex Message 美化排行榜版面，提升使用者閱讀與互動體驗。

由於 LINE Carousel Template 存在每次僅能顯示 10 張專輯的限制，當專輯總數超過 10 張時，最後一張專輯將無法正常顯示，亦無法提供完整投票與資訊查詢功能。後續可考慮導入 Carousel 分頁邏輯或改用 LIFF 網頁呈現完整專輯清單，以解決此上限問題。

目前所有專輯皆已補齊正式封面圖片，整體視覺完整性與呈現效果已大幅提升。另一方面，投票資料目前僅儲存在記憶體中，當服務中斷後資料即會消失，尚未實作資料持久性儲存功能。未來可考慮接入雲端資料庫以保存使用者歷史紀錄。

此外，針對新進使用者，目前系統尚未提供歡迎訊息或基本使用教學，可能導致初次使用者對操作流程感到迷惘。後續可於初次互動時設計歡迎提示與簡易功能導覽，提升系統易用性與友善度。

在持續開發方向上，未來可新增 Flex Message 排行榜版面設計、使用者個人投票紀錄查詢功能，並可進一步支援模糊歌曲查詢（如 partial match、fuzzy matching）以強化搜尋彈性。此外，也可整合 LIFF 前端互動式選單，提供圖形化操作介面，並將系統部署至 Render 或 Hugging Face 等雲端平台，以達成 24/7 穩定運作之完整服務架構。

**六、整體專案收穫**

完整整合以下技能模組：
- LINE Messaging API 與 Webhook 設計
- Carousel UI 建置
- 反向字典查詢資料結構
- pyngrok + Colab 整合部署
- APIClient 新版 SDK 實作
- 憑證安全管理設計